# Recupération et transformation en valeurs numériques de la data base

In [37]:
import pandas as pd
import numpy as np
df = pd.read_csv("../4072eb5e-e963-4a17-a794-3ea028d0a9c4.csv")
df.shape

(1338, 7)

In [38]:
df.drop_duplicates(inplace=True)
df.reset_index(inplace=True)
df.head()

,index,age,sex,bmi,children,smoker,region,charges
0,0,19,female,27.900,0,yes,southwest,16884.92400
1,1,18,male,33.770,1,no,southeast,1725.55230
2,2,28,male,33.000,3,no,southeast,4449.46200
3,3,33,male,22.705,0,no,northwest,21984.47061
4,4,32,male,28.880,0,no,northwest,3866.85520


In [39]:
df["sex"] = df["sex"].replace(["female","male"],[0,1])
df["smoker"] = df["smoker"].replace(["no","yes"],[0,1])
df.head()

/tmp/ipykernel_21213/259323464.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df["sex"] = df["sex"].replace(["female","male"],[0,1])
/tmp/ipykernel_21213/259323464.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df["smoker"] = df["smoker"].replace(["no","yes"],[0,1])


,index,age,sex,bmi,children,smoker,region,charges
0,0,19,0,27.900,0,1,southwest,16884.92400
1,1,18,1,33.770,1,0,southeast,1725.55230
2,2,28,1,33.000,3,0,southeast,4449.46200
3,3,33,1,22.705,0,0,northwest,21984.47061
4,4,32,1,28.880,0,0,northwest,3866.85520


In [40]:
one_hot = pd.get_dummies(df["region"], dtype="int64")
df.drop("region", axis=1, inplace=True)
df = df.join(one_hot)

In [41]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1337 entries, 0 to 1336
Data columns (total 11 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   index      1337 non-null   int64  
 1   age        1337 non-null   int64  
 2   sex        1337 non-null   int64  
 3   bmi        1337 non-null   float64
 4   children   1337 non-null   int64  
 5   smoker     1337 non-null   int64  
 6   charges    1337 non-null   float64
 7   northeast  1337 non-null   int64  
 8   northwest  1337 non-null   int64  
 9   southeast  1337 non-null   int64  
 10  southwest  1337 non-null   int64  
dtypes: float64(2), int64(9)
memory usage: 115.0 KB


# Méthode des moindres carrés

On commence par définir notre matrice X des valeurs explicatives et notre matrice target Y 

In [42]:
Y = df["charges"]
X = df.drop("charges",axis=1)
X

,index,age,sex,bmi,children,smoker,northeast,northwest,southeast,southwest
0,0,19,0,27.900,0,1,0,0,0,1
1,1,18,1,33.770,1,0,0,0,1,0
2,2,28,1,33.000,3,0,0,0,1,0
3,3,33,1,22.705,0,0,0,1,0,0
4,4,32,1,28.880,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...
1332,1333,50,1,30.970,3,0,0,1,0,0
1333,1334,18,0,31.920,0,0,1,0,0,0
1334,1335,18,0,36.850,0,0,0,0,1,0
1335,1336,21,0,25.800,0,0,0,0,0,1


On rajoute une ligne de 1 à notre matrice X pour la constante (équivalent de l'ordonnée à l'origine) 
Le problème étant que la matrice que l'on n'obtient ainsi n'est pas inversible... On va donc s'en passer pour utiliser la méthode des moindres carrés.

In [43]:
# X['constant'] = X.apply(lambda _: 1, axis=1)
# column_to_move = X.pop("constant")
# X.insert(0, "constant", column_to_move)
# X

Le calcul des poids'obtient ensuite avec la formule : $$B = (X^TX)^{-1}X^TY$$

In [44]:
XT = X.transpose()
B = np.linalg.inv(XT@X)@XT@Y
B.to_numpy()

array([ 2.04196772e-01,  2.56930643e+02, -1.30136231e+02,  3.39633230e+02,
        4.73137750e+02,  2.38462680e+04, -1.20877586e+04, -1.24405189e+04,
       -1.31222350e+04, -1.30500477e+04])

On obtient ensuite nos solutions éstimées : $$Y'=(XB)$$

In [45]:
Y_estim = X.to_numpy() @(B)
Y_estim

array([25153.6696415 ,  3315.1364516 ,  6569.40498847, ...,
        4290.60377589,  1380.84003828, 37224.66736046], shape=(1337,))

On peux maintenant calculer notre RMSE, pour cela on a besoin de la Somme des Carrés Résiduelle SCR $$\sum_{i=1}^{n} (y_{i}-y'_{i})^2$$

In [48]:
SCR = 0
n = Y.shape[0]

for i in range(n):
    SCR += ((Y[i] - Y_estim[i])**2)

np.sqrt(SCR/n)

np.float64(6043.336596262366)

Notre RMSE est égale à 6043 ce qui n'est pas fou mais pas si mal vu la simplicité de la méthode...

# ANOVA et calcul du coefficient de détermination

In [51]:
SCT = 0
SCE = 0
n = Y.shape[0]
Y_mean = Y.mean()
for i in range(n):
    SCT += ((Y[i] - Y_mean)**2)
    SCE += ((Y_estim[i] - Y_mean)**2)

R2=SCE/SCT
R2

np.float64(0.750790591369294)

# Test de significativité

## Global